In [1]:
import pandas as pd
import io
from tqdm import tqdm
from time import sleep
import numpy as np
import requests
import math
import json

In [2]:
filepath = '../data/NAD_r11.txt' # too large to push onto Git, but is part of the download from here: https://nationaladdressdata.s3.amazonaws.com/NAD_r12_TXT.zip

In [3]:
first_line = None

with open(filepath) as f:
    first_line = f.readline()

print(first_line)

OID,State,County,Inc_Muni,Uninc_Comm,Nbrhd_Comm,Post_Comm,Zip_Code,Plus_4,Bulk_Zip,Bulk_Plus4,StN_PreMod,StN_PreDir,StN_PreTyp,StN_PreSep,StreetName,StN_PosTyp,StN_PosDir,StN_PosMod,AddNum_Pre,Add_Number,AddNum_Suf,LandmkPart,LandmkName,Building,Floor,Unit,Room,Addtl_Loc,Milepost,Longitude,Latitude,NatGrid_Coord,GUID,Addr_Type,Placement,Source,AddAuth,UniqWithin,LastUpdate,Effective,Expired



In [4]:
total_lines = sum(1 for line in open(filepath))
print('Total number of lines: %s' % total_lines)

Total number of lines: 67357690


In [5]:
import warnings
warnings.filterwarnings('ignore')

In [27]:
def check_int(val):
    num = None
    try:
        num = str(int(float(val)))
    except Exception as e: print(e)
    return num

In [29]:
step = 1000
dfs = []
i = 0
# total_lines = 100 # for checking

with tqdm(total=total_lines) as pbar:
    with open(filepath) as f:
        f.readline() # skip first line
        lines = ''
        for line in f:
            pbar.update(1)
            i += 1
            lines += line
                        
            if i > step:
                df = pd.read_csv(io.StringIO(first_line + lines), low_memory=False)
                df = df[['State', 'County', 'Zip_Code', 'Post_Comm', 'Add_Number', 'StN_PreMod', 'StN_PreDir', 'StN_PreTyp', 'StN_PreSep', 'StreetName', 'StN_PosTyp', 'StN_PosDir', 'StN_PosMod', 'Longitude', 'Latitude']]
                df = df.astype(str)   
                
                # run cleaning and filters here
                df = df.replace('nan', '')
                df = df.fillna('')   
                df = df[df['Zip_Code'] != 'nan']
                df = df[df['Zip_Code'] != '']
                df = df.apply(lambda x: x.astype(str).str.lower()) # convert everything lower case
                
                # cleaning zip code
                df['Zip_Code'] = df['Zip_Code'].apply(lambda x: check_int(x))
                
                # removing invalid zip rows
                df = df[df['Zip_Code'].notnull()]
                
                if df.empty: # don't append it and go next
                    i = 0
                    lines = '' 
                    continue
                
                # print(df['Zip_Code'].unique())
                # input('test')
                
                # constructing address
                df['Prefix'] = df['Add_Number'] + ' ' + df['StN_PreMod'] + ' '+ df['StN_PreDir'] + ' '+ df['StN_PreTyp'] + ' '+ df['StN_PreSep'] + ' '+ df['StreetName'] + ' '+ df['StN_PosTyp'] + ' '+ df['StN_PosDir'] + ' '+ df['StN_PosMod']
                df['Suffix'] = ','+df['Post_Comm'] + ','+ df['State'] + ',' +df['Zip_Code']
                df['Prefix'] = df['Prefix'].apply(lambda x: " ".join(x.split()))
                df['Suffix'] = df['Suffix'].apply(lambda x: " ".join(x.split()))
                df['Address'] = df['Prefix'] + df['Suffix']    
                
                # rename the columns
                df.rename(columns={
                    'State': 'state', 
                    'County': 'county', 
                    'Zip_Code': 'zip',
                    'Add_Number': 'add_number',
                    'StreetName': 'street_name',
                    'Longitude': 'longitude',
                    'Latitude': 'latitude',
                    'Address': 'address',
                }, inplace=True)
                
                # Remove unecessary columns
                df = df[['state', 'county', 'zip', 'longitude', 'latitude', 'address']]
                
                # Store this partial data frame
                dfs.append(df)
                i = 0
                lines = ''             
                

  5%|███████▋                                                                                                                                                 | 3410408/67357690 [01:23<24:59, 42641.30it/s]

could not convert string to float: 'ia'


  6%|█████████▍                                                                                                                                               | 4170167/67357690 [01:41<24:35, 42834.55it/s]

could not convert string to float: '5-156'


 17%|██████████████████████████▏                                                                                                                             | 11615605/67357690 [04:29<20:53, 44458.68it/s]

...

 98%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋   | 65880816/67357690 [26:05<00:31, 46303.97it/s]

could not convert string to float: 'acme'
could not convert string to float: 'acme'
could not convert string to float: 'acme'
could not convert string to float: 'acme'


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 67083017/67357690 [26:32<00:06, 44380.81it/s]

could not convert string to float: 'out'
could not convert string to float: 'out'


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 67357689/67357690 [26:37<00:00, 42166.42it/s]


In [30]:
# Column descriptors: https://www.transportation.gov/sites/dot.gov/files/docs/mission/gis/national-address-database/308816/nad-schema-v1.pdf

In [31]:
len(dfs)

63872

In [32]:
df = pd.concat(dfs)

In [33]:
df

,state,county,zip,longitude,latitude,address
0,az,coconino,86336,-111.771770637,34.8280627940001,"95 lynx drive,sedona,az,86336"
1,az,coconino,86336,-111.769395183,34.828455976,"33 eagle lane,sedona,az,86336"
2,az,coconino,86336,-111.777281707,34.828623958,"141 vista bonita drive,sedona,az,86336"
3,az,coconino,86336,-111.775765885,34.8291778600001,"109 badger drive,sedona,az,86336"
4,az,coconino,86336,-111.773383133,34.824991696,"30 fawn spur,sedona,az,86336"
...,...,...,...,...,...,...
104,pa,westmoreland,15601,-79.52236881714968,40.290077006924825,"605 farview drive,greensburg,pa,15601"
105,pa,westmoreland,15601,-79.52241451804144,40.29008212001564,"607 farview drive,greensburg,pa,15601"
106,pa,westmoreland,15601,-79.52211734366738,40.290106697381,"597 farview drive,greensburg,pa,15601"
553,pa,westmoreland,15601,-79.52401315183624,40.29069422586329,"598 farview drive,greensburg,pa,15601"


# Sample verify address

In [34]:
# url = 'https://geocoding.geo.census.gov/geocoder/locations/addressbatch'
# s_buf = io.StringIO()
# # saving a data frame to a buffer (same as with a regular file):
# # s = '\n'.join(list(df['Address']))
# # print(s)
# s_buf = io.StringIO()
# # saving a data frame to a buffer (same as with a regular file):
# df['Address'].to_csv(s_buf, index=False)
# df['Address'].to_csv('../data/temp_addr.csv', index=False)
# print(df['Address'])

In [35]:
test_address = df['address'].sample(1).values[0]
print(test_address)
r = requests.get('https://geocoding.geo.census.gov/geocoder/locations/onelineaddress?address=%s&benchmark=2020&format=json' % test_address).json()
print(r)  
print(len(r['result']['addressMatches']) > 0)

4992 pedersen road,batavia,oh,45103
{'result': {'input': {'address': {'address': '4992 pedersen road,batavia,oh,45103'}, 'benchmark': {'isDefault': False, 'benchmarkDescription': 'Public Address Ranges - Census 2020 Benchmark', 'id': '2020', 'benchmarkName': 'Public_AR_Census2020'}}, 'addressMatches': [{'tigerLine': {'side': 'R', 'tigerLineId': '649136393'}, 'coordinates': {'x': -84.1096450115893, 'y': 39.17269972282121}, 'addressComponents': {'zip': '45103', 'streetName': 'PEDERSEN', 'preType': '', 'city': 'BATAVIA', 'preDirection': '', 'suffixDirection': '', 'fromAddress': '4800', 'state': 'OH', 'suffixType': 'RD', 'toAddress': '5998', 'suffixQualifier': '', 'preQualifier': ''}, 'matchedAddress': '4992 PEDERSEN RD, BATAVIA, OH, 45103'}]}}
True


In [36]:
# # https://geocoding.geo.census.gov/geocoder/Geocoding_Services_API.pdf
# # curl --form addressFile=@localfile.csv --form benchmark=2020 https://geocoding.geo.census.gov/geocoder/locations/addressbatch --output geocoderesult.csv
# # https://curlconverter.com/

# url = 'https://geocoding.geo.census.gov/geocoder/locations/addressbatch'
# files = {
#     'addressFile': open('../data/temp_addr.csv', 'r'),
#     'benchmark': (None, '2020'),
# }

# response = requests.post('https://geocoding.geo.census.gov/geocoder/locations/addressbatch', files=files)
# print(response.content)
# print(response)

In [37]:
# len(df['State'].unique()) # check number of different states

# Save files by county (because by zip code > 50 Mb file limit of GitHub)

In [38]:
counties = df['county'].unique()
sc_list = []

for county in tqdm(counties):
    pdf = df[df['county'] == county]
    state = list(pdf['state'].unique())[0] # get the state value
    pdf.to_csv('../data/%s_%s.csv.xz' % (state, county), index=False)
    sc_list.append('%s_%s.csv.xz' % (state, county))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1082/1082 [3:00:44<00:00, 10.02s/it]


# Save list of names to a json file

In [39]:
with open('../data/state_county.json', 'w') as f:
    json.dump(sc_list, f)